# Problem

Interpret a 0 Layer transformer trained to predict the next token on a tiny dataset (see 'Task Description' section for full dataset).

Concretely, a solution looks like:
1. Describing a mechanism for how the model is able to minimize loss when predicting the next token. Ideally your explanation should include an interpretation of the model weights.
2. Providing evidence that this interpretation is actually true, ideally using some mechanistic interpretability technique(s). Hint: This model and dataset are small enough that you should be able to understand all of the weights.

Since this is the first problem in this sequence, feel free to jump straight to the solution to get a feel of what a solution to these problems looks like. However I still recommend at least poking around the model to build some intuition / guesses before doing this.

# Setup
(No need to read)

In [ ]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-x7il69mg
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-x7il69mg
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 10d2f8a026d73eada861c7d51064f7e24d8f482c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
try:
    %pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
except:
    import os; os.environ["ACCELERATE_DISABLE_RICH"] = "1"
    from IPython import get_ipython
    ipython = get_ipython()
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-75pvqurp
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-75pvqurp
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit cc216772a66af819ff3a77038e53134f3e073af4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for circuitsvis: filename=circuitsvis-0.0.0-py3-none-any.whl size=1808565 sha256=00f6ea06383f60ca2f217e993df066f72a161cc8394eab833ee3e1580b220973
  Stored in directory: /tmp/pip-ephem-wheel-cache-maxmeisg/wheels/86/be/ad/78078aba9344d200aad61b63d35cdaecdec160212f039eed74
Successfully built circuitsvis
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import circuitsvis as cv
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
from dataclasses import dataclass
import datasets
from IPython.display import HTML

In [ ]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [ ]:
import plotly.graph_objects as go

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth"}
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    if isinstance(tensor, list):
        tensor = torch.stack(tensor)
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0,labels={"x":xaxis, "y":yaxis}, **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label

    fig.show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(y=utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    # Helper function to plot multiple lines
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

def bar(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.bar(
        y=utils.to_numpy(tensor),
        labels={"x": xaxis, "y": yaxis},
        template="simple_white",
        **kwargs).show(renderer)

In [ ]:
import transformer_lens.patching as patching
from transformer_lens import evals
import math

In [ ]:
def disable_biases(model):
    for name, param in model.named_parameters():
        if 'b_' in name:
            param.requires_grad = False

def disable_pos_embed(model):
    assert model.cfg.positional_embedding_type == "standard"
    model.pos_embed.W_pos = nn.Parameter(torch.zeros_like(model.pos_embed.W_pos))
    model.pos_embed.W_pos.requires_grad = False

# Task Description

In this notebook I train 0L transformer to predict the next token on a tiny corpus of data. Your job is to reverse engineer how the model does this.

The dataset is extremely tiny, which is unrealistic, but makes the analysis way easier. Below 'tokens' is the full dataset that we will use to train this model.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
text = """Before moving on to more complex models, it’s useful to briefly consider a zero-layer transformer. Such a model takes a token, embeds it, unembeds it to produce logits predicting the next token"""
vocab = set(text)
print(vocab)
char_to_token = {ch: i for i, ch in enumerate(vocab)}
print(char_to_token)
tokens = torch.tensor([char_to_token[ch] for ch in text]).unsqueeze(0).to(device)
print(tokens.shape)
print(tokens)

{'s', 'k', 'n', ' ', 'o', 'v', 'm', 'x', 'e', 'p', '.', 'S', 'g', 'u', ',', 'y', 'i', 'b', 'c', 'B', 't', 'a', '-', 'l', 'd', 'f', '’', 'r', 'z', 'h'}
{'s': 0, 'k': 1, 'n': 2, ' ': 3, 'o': 4, 'v': 5, 'm': 6, 'x': 7, 'e': 8, 'p': 9, '.': 10, 'S': 11, 'g': 12, 'u': 13, ',': 14, 'y': 15, 'i': 16, 'b': 17, 'c': 18, 'B': 19, 't': 20, 'a': 21, '-': 22, 'l': 23, 'd': 24, 'f': 25, '’': 26, 'r': 27, 'z': 28, 'h': 29}
torch.Size([1, 193])
tensor([[19,  8, 25,  4, 27,  8,  3,  6,  4,  5, 16,  2, 12,  3,  4,  2,  3, 20,
          4,  3,  6,  4, 27,  8,  3, 18,  4,  6,  9, 23,  8,  7,  3,  6,  4, 24,
          8, 23,  0, 14,  3, 16, 20, 26,  0,  3, 13,  0,  8, 25, 13, 23,  3, 20,
          4,  3, 17, 27, 16,  8, 25, 23, 15,  3, 18,  4,  2,  0, 16, 24,  8, 27,
          3, 21,  3, 28,  8, 27,  4, 22, 23, 21, 15,  8, 27,  3, 20, 27, 21,  2,
          0, 25,  4, 27,  6,  8, 27, 10,  3, 11, 13, 18, 29,  3, 21,  3,  6,  4,
         24,  8, 23,  3, 20, 21,  1,  8,  0,  3, 21,  3, 20,  4,  1,  8,  2, 14,


# Load Model

A 0L transformer just takes a token, embeds it, unembeds it to produce logits predicting the next token:

In [ ]:
torch.set_grad_enabled(True)

In [ ]:
@dataclass
class Config:
    d_vocab: int = len(vocab)
    d_model: int = 16
    seed = 0

cfg = Config()
print(cfg)

Config(d_vocab=30, d_model=16)


In [ ]:
class ZeroLayerTransformer(HookedRootModule):
    def __init__(self, cfg):
        super().__init__()
        torch.manual_seed(cfg.seed)
        self.W_E = nn.Parameter(torch.randn((cfg.d_vocab, cfg.d_model)) / np.sqrt(cfg.d_model))
        self.W_U = nn.Parameter(torch.randn((cfg.d_model, cfg.d_vocab)) / np.sqrt(cfg.d_model))

        self.hook_embed = HookPoint()
        super().setup()

    def forward(self, x):
        embed = self.hook_embed(self.W_E[x])
        return embed @ self.W_U

model = ZeroLayerTransformer(cfg).to(device)
print(model)

ZeroLayerTransformer(
  (hook_embed): HookPoint()
)


In [ ]:
for name, param in model.named_parameters():
    print(name, param.shape, param.requires_grad)

W_E torch.Size([30, 16]) True
W_U torch.Size([16, 30]) True


# Training

Training only takes ~10 seconds to converge, so we just train in this notebook:

## Loss Fn

The model is trained to minimize cross entropy loss:

In [ ]:
def loss_fn(logits, tokens):
    logits = logits[:, :-1]
    labels = tokens[:, 1:]

    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[..., None])[..., 0]
    return -correct_log_probs.mean()

with torch.inference_mode():
    logits = model(tokens)
    loss = loss_fn(logits, tokens)
    print(loss)

tensor(3.4703, device='cuda:0')


In [ ]:
print("uniform loss:", np.log(cfg.d_vocab))

uniform loss: 3.4011973816621555


## Setup optimizer

In [ ]:
lr = 1e-3
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

## Training Loop

In [ ]:
num_epochs = 10_000

train_losses = []
for epoch in tqdm.tqdm(range(num_epochs)):
    logits = model(tokens)
    loss = loss_fn(logits, tokens)
    loss.backward()
    train_losses.append(loss.item())

    optimizer.step()
    optimizer.zero_grad()

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, train loss: {loss.item()}")

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 0, train loss: 3.4702975749969482
Epoch: 100, train loss: 3.1267600059509277
Epoch: 200, train loss: 2.7095038890838623
Epoch: 300, train loss: 2.3034093379974365
Epoch: 400, train loss: 2.0350890159606934
Epoch: 500, train loss: 1.8763816356658936
Epoch: 600, train loss: 1.7787609100341797
Epoch: 700, train loss: 1.718766212463379
Epoch: 800, train loss: 1.682808756828308
Epoch: 900, train loss: 1.660979151725769
Epoch: 1000, train loss: 1.6471027135849
Epoch: 1100, train loss: 1.6378393173217773
Epoch: 1200, train loss: 1.6313813924789429
Epoch: 1300, train loss: 1.626711130142212
Epoch: 1400, train loss: 1.6232284307479858
Epoch: 1500, train loss: 1.6205635070800781
Epoch: 1600, train loss: 1.618479609489441
Epoch: 1700, train loss: 1.6168196201324463
Epoch: 1800, train loss: 1.6154756546020508
Epoch: 1900, train loss: 1.6143726110458374
Epoch: 2000, train loss: 1.6134560108184814
Epoch: 2100, train loss: 1.6126861572265625
Epoch: 2200, train loss: 1.6120336055755615
Epoch: 2

In [ ]:
line(
    train_losses,
    title="Loss Curve",
    xaxis="Epoch", yaxis="Loss"
)